<a href="https://colab.research.google.com/github/egonik-unlp/tp_curso_ml/blob/main/src/notebooks/Autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.preprocessing import normalize, MinMaxScaler
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/protein_classifier_with_locations/dataset_proteoma_full/
proteome=np.load("proteome_short.npy")

/content/drive/MyDrive/protein_classifier_with_locations/dataset_proteoma_full


In [18]:
norm=X_train.max()

In [3]:
scaler=MinMaxScaler()
X_train_val, X_test=train_test_split(proteome)
X_train, X_valid=train_test_split(X_train_val)

X_train_n=scaler.fit_transform(X_train)
X_valid_n=scaler.transform(X_valid)

X_train_n=X_train/X_train.max()
X_valid

In [19]:
X_test_n=scaler.transform(X_test)

In [4]:
X_train_bis=X_train_n.reshape(-1,2000,1)
X_valid_bis=X_valid_n.reshape(-1,2000,1)


In [5]:
tf.keras.backend.clear_session()

conv_encoder = keras.models.Sequential([
keras.layers.Reshape([2000, 1], input_shape=[2000,1]),
keras.layers.Conv1D(16, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2),
keras.layers.Conv1D(32, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2),
keras.layers.Conv1D(64, kernel_size=3, padding="same", activation="selu"),
keras.layers.MaxPool1D(pool_size=2)
])


conv_decoder = keras.models.Sequential([
keras.layers.Conv1DTranspose(32, kernel_size=3, strides=2, padding="same",
activation="selu",
input_shape=[250, 64]),
keras.layers.Conv1DTranspose(16, kernel_size=3, strides=2, padding="same",
activation="selu"),
keras.layers.Conv1DTranspose(1, kernel_size=3, strides=2, padding="same",
activation="sigmoid"),

])


conv_ae = keras.models.Sequential([conv_encoder, conv_decoder])
conv_ae.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (None, 250, 64)           7840      
_________________________________________________________________
sequential_1 (Sequential)    (None, 2000, 1)           7777      
Total params: 15,617
Trainable params: 15,617
Non-trainable params: 0
_________________________________________________________________


In [59]:
conv_ae.compile(loss="binary_crossentropy",
optimizer=keras.optimizers.SGD(learning_rate=1.5))
history = conv_ae.fit(X_train_bis, X_train_bis, epochs=10,
validation_data=[X_valid_bis, X_valid_bis])

Epoch 1/10
1758/1758 [==============================] - 21s 11ms/step - loss: 0.0800 - val_loss: 0.0000e+00
Epoch 2/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0800 - val_loss: 0.0000e+00
Epoch 3/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0799 - val_loss: 0.0000e+00
Epoch 4/10
1758/1758 [==============================] - 20s 11ms/step - loss: 0.0799 - val_loss: 0.0000e+00
Epoch 5/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0799 - val_loss: 0.0000e+00
Epoch 6/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0798 - val_loss: 0.0000e+00
Epoch 7/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0798 - val_loss: 0.0000e+00
Epoch 8/10
1758/1758 [==============================] - 19s 11ms/step - loss: 0.0797 - val_loss: 0.0000e+00
Epoch 9/10
1758/1758 [==============================] - 20s 11ms/step - loss: 0.0797 - val_loss: 0.0000e+00
Epoch 10/10
1758/1758 [=====

In [7]:
conv_encoder.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 2000, 1)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 2000, 16)          64        
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1000, 16)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1000, 32)          1568      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 500, 32)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 64)           6208      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 250, 64)           0

In [8]:
conv_decoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_transpose (Conv1DTran (None, 500, 32)           6176      
_________________________________________________________________
conv1d_transpose_1 (Conv1DTr (None, 1000, 16)          1552      
_________________________________________________________________
conv1d_transpose_2 (Conv1DTr (None, 2000, 1)           49        
Total params: 7,777
Trainable params: 7,777
Non-trainable params: 0
_________________________________________________________________


In [60]:
predictions=conv_ae.predict(X_test_n)
pred_num=scaler.inverse_transform(predictions.reshape(-1,2000))

In [ ]:
pred_num_rounded=np.vectorize(round)(pred_num)

In [ ]:
np.save("autoencoded_sequences",pred_num_rounded)

In [ ]:
seq110=X_test[110][np.where(X_test[110]!=0)]

In [ ]:
seq110

In [ ]:
pred_110=pred_num_rounded[110,(np.where(pred_num_rounded[110]!= 0 ))].flatten()

In [ ]:
sum(np.equal(seq110,pred_110[1:]))/len(seq110)